In [ ]:
!pip install protobuf transformers==4.28.0 cpm_kernels torch>=1.10 gradio mdtex2html sentencepiece accelerate

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import json
# GPU设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 加载模型与tokenizer
model_name_or_path = 'scutcyr/SoulChat'
model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True).half()
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

def get_dialogue_history(dialogue_history_list: list):

    dialogue_history_tmp = []
    for item in dialogue_history_list:
        if item['role'] == 'counselor':
            text = '心理咨询师：'+ item['content']
        else:
            text = '用户：'+ item['content']
        dialogue_history_tmp.append(text)

    dialogue_history = '\n'.join(dialogue_history_tmp)

    return dialogue_history + '\n' + '心理咨询师：'

with open('./auto_eval_chat_data.json') as f:
    data = json.load(f)

    for i in range(100):
        print(f'i'.center(50, '*'))
        dialogue_history_with_label = []
        dialogue_history_with_output = []

        dialogue = data[i]['messages']
        for j in range(0,10,2):
            usr_msg = dialogue[j]['content']
            dialogue_history_with_label.append({
                'role': 'client',
                'content': usr_msg
            })
            dialogue_history_with_output.append({
                'role': 'client',
                'content': usr_msg
            })

            input_text = get_dialogue_history(dialogue_history_list=dialogue_history_with_output)
            response, history = model.chat(tokenizer, query=input_text, history=None, max_length=2048, num_beams=1, do_sample=True, top_p=0.75, temperature=0.95, logits_processor=None)
            dialogue_history_with_output.append({
                'role': 'counselor',
                'content': response
            })
            print(dialogue_history_with_output)
            dialogue[j+1]['SoulChat_response_history_with_output'] = response

            input_text = get_dialogue_history(dialogue_history_list=dialogue_history_with_label)
            # instruction = get_instruction(dialogue_history=dialogue_history)
            response, history = model.chat(tokenizer, query=input_text, history=None, max_length=2048, num_beams=1, do_sample=True, top_p=0.75, temperature=0.95, logits_processor=None)
            dialogue_history_with_label.append({
                'role': 'counselor',
                'content': dialogue[j+1]['content']
            })
            dialogue[j+1]['SoulChat_response_history_with_label'] = response

with open('./SoulChat_result.json', 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)